<a href="https://colab.research.google.com/github/WISEPLAT/_NN/blob/main/3_%D1%82%D0%BE%D0%BD%D0%BA%D0%B0%D1%8F_%D0%BD%D0%B0%D1%81%D1%82%D1%80%D0%BE%D0%B9%D0%BA%D0%B0_%D0%BD%D0%B5%D0%B9%D1%80%D0%BE%D0%BD%D0%BD%D0%BE%D0%B9_%D1%81%D0%B5%D1%82%D0%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from keras.datasets import mnist # subroutines for fetching the MNIST dataset
from keras.models import Model # basic class for specifying and training a neural network
from keras.layers import Input, Dense, Flatten, Convolution2D, MaxPooling2D, Dropout
from keras.utils import np_utils # utilities for one-hot encoding of ground truth values

In [ ]:
batch_size = 128 # in each iteration, we consider 128 training examples at once
num_epochs = 12 # we iterate twelve times over the entire training set
kernel_size = 3 # we will use 3x3 kernels throughout
pool_size = 2 # we will use 2x2 pooling throughout
conv_depth = 32 # use 32 kernels in both convolutional layers
drop_prob_1 = 0.25 # dropout after pooling with probability 0.25
drop_prob_2 = 0.5 # dropout in the FC layer with probability 0.5
hidden_size = 128 # there will be 128 neurons in both hidden layers

In [ ]:
num_train = 60000 # there are 60000 training examples in MNIST
num_test = 10000 # there are 10000 test examples in MNIST

height, width, depth = 28, 28, 1 # MNIST images are 28x28 and greyscale
num_classes = 10 # there are 10 classes (1 per digit)

(X_train, y_train), (X_test, y_test) = mnist.load_data() # fetch MNIST data

X_train = X_train.reshape(X_train.shape[0], depth, height, width)
X_test = X_test.reshape(X_test.shape[0], depth, height, width)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255 # Normalise data to [0, 1] range
X_test /= 255 # Normalise data to [0, 1] range

Y_train = np_utils.to_categorical(y_train, num_classes) # One-hot encode the labels
Y_test = np_utils.to_categorical(y_test, num_classes) # One-hot encode the labels

In [ ]:
inp = Input(shape=(depth, height, width)) # N.B. Keras expects channel dimension first
# Conv [32] -> Conv [32] -> Pool (with dropout on the pooling layer)
conv_1 = Convolution2D(conv_depth, kernel_size, kernel_size, padding='same', activation='relu')(inp)
conv_2 = Convolution2D(conv_depth, kernel_size, kernel_size, padding='same', activation='relu')(conv_1)
pool_1 = MaxPooling2D(pool_size=(pool_size, pool_size), padding='same')(conv_2)
drop_1 = Dropout(drop_prob_1)(pool_1)
flat = Flatten()(drop_1)
hidden = Dense(hidden_size, activation='relu')(flat) # Hidden ReLU layer
drop = Dropout(drop_prob_2)(hidden)
out = Dense(num_classes, activation='softmax')(drop) # Output softmax layer

model = Model(inputs=inp, outputs=out) # To define a model, just specify its input and output layers

model.compile(loss='categorical_crossentropy', # using the cross-entropy loss function
              optimizer='adam', # using the Adam optimiser
              metrics=['accuracy']) # reporting the accuracy

In [ ]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1, 28, 28)]       0         
                                                                 
 conv2d_4 (Conv2D)           (None, 1, 10, 32)         8096      
                                                                 
 conv2d_5 (Conv2D)           (None, 1, 4, 32)          9248      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 1, 2, 32)         0         
 2D)                                                             
                                                                 
 dropout_4 (Dropout)         (None, 1, 2, 32)          0         
                                                                 
 flatten_2 (Flatten)         (None, 64)                0         
                                                           

In [ ]:
model.fit(X_train, Y_train, # Train the model using the training set...
          batch_size=batch_size, epochs=num_epochs,
          verbose=1, validation_split=0.1) # ...holding out 10% of the data for validation

Epoch 1/12
422/422 [==============================] - 3s 7ms/step - loss: 0.8755 - accuracy: 0.7160 - val_loss: 0.2259 - val_accuracy: 0.9398
Epoch 2/12
422/422 [==============================] - 3s 6ms/step - loss: 0.3834 - accuracy: 0.8855 - val_loss: 0.1546 - val_accuracy: 0.9582
Epoch 3/12
422/422 [==============================] - 3s 6ms/step - loss: 0.2955 - accuracy: 0.9133 - val_loss: 0.1250 - val_accuracy: 0.9662
Epoch 4/12
422/422 [==============================] - 3s 6ms/step - loss: 0.2563 - accuracy: 0.9244 - val_loss: 0.1129 - val_accuracy: 0.9703
Epoch 5/12
422/422 [==============================] - 2s 6ms/step - loss: 0.2280 - accuracy: 0.9321 - val_loss: 0.1001 - val_accuracy: 0.9725
Epoch 6/12
422/422 [==============================] - 3s 6ms/step - loss: 0.2106 - accuracy: 0.9373 - val_loss: 0.0930 - val_accuracy: 0.9733
Epoch 7/12
422/422 [==============================] - 3s 6ms/step - loss: 0.1953 - accuracy: 0.9425 - val_loss: 0.0879 - val_accuracy: 0.9742
Epoch 

In [ ]:
model.evaluate(X_test, Y_test, verbose=1) # Evaluate the trained model on the test set!

313/313 [==============================] - 0s 1ms/step - loss: 0.0822 - accuracy: 0.9732


[0.08220253884792328, 0.9732000231742859]